In [ ]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## 각자 드라이브 위치로 경로를 잘 바꿔주세요~
# data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data')
train_path = "D:\\Users\\tonyn\\Desktop\\da_sci_4th\\recomendation_system\\rec_data\\rec_data\\ratings.dat"
train_path

'D:\\Users\\tonyn\\Desktop\\da_sci_4th\\recomendation_system\\rec_data\\rec_data\\ratings.dat'

In [ ]:
def load_data(data_path: Path):
    data = pd.read_csv( 
        data_path, 
        sep='::', 
        header=None, 
        usecols=[0, 1, 2, 3], 
        dtype={0: np.int32, 1: np.int32, 2: np.int32}, 
    )
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    # time 형식을 기존과 맞게 고쳐보기
    data['Time'] = pd.to_datetime(data['Time'], unit='s')
    return data

data = load_data(train_path)
data.sort_values(['UserId', 'Time'], inplace=True)
data

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


In [ ]:
# short_session을 제거한 다음 unpopular item을 제거하면 다시 길이가 1인 session이 생길 수 있습니다.
# 이를 위해 반복문을 통해 지속적으로 제거 합니다.
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('UserId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['UserId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [ ]:
data = cleanse_recursive(data, shortest=2, least_click=5)
data

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


In [ ]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    cutoff_time = final_time - dt.timedelta(days=n_days)

    # Train: cutoff 이전의 데이터
    train = data[data['Time'] < cutoff_time]

    # Test: cutoff 이후의 데이터, 단 아이템은 train에 있던 것만
    test = data[(data['Time'] >= cutoff_time) & (data['ItemId'].isin(train['ItemId']))]

    return train, test

In [ ]:
tr, test = split_by_date(data, n_days=100)  
tr, val = split_by_date(tr, n_days=100)  

In [ ]:
# data에 대한 정보를 살펴봅니다.
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [ ]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 989344
	 Sessions: 6039
	 Items: 3416
	 First Time : 2000-04-25 23:05:32
	 Last Time : 2002-08-12 13:53:29

* valid Set Stats Info
	 Events: 5200
	 Sessions: 245
	 Items: 1940
	 First Time : 2002-08-13 00:40:06
	 Last Time : 2002-11-20 16:38:40

* test Set Stats Info
	 Events: 5067
	 Sessions: 254
	 Items: 1887
	 First Time : 2002-11-20 20:30:02
	 Last Time : 2003-02-28 17:49:50



In [ ]:
tr.shape
tr

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


In [ ]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [ ]:
# tr, val, test에 대해 item_idx가 -1인 행을 제거합니다.
tr = tr[tr['item_idx'] != -1]
val = val[val['item_idx'] != -1]
test = test[test['item_idx'] != -1]

In [ ]:
#-1인 이벤트 제거 후, 다시 최소 세션 길이를 보장하기 위해 cleanse_recursive 적용
# least_click은 0으로 설정하여 아이템 인기도 기준은 적용하지 않습니다.
tr = cleanse_recursive(tr, shortest=2, least_click=0) 
val = cleanse_recursive(val, shortest=2, least_click=0)
test = cleanse_recursive(test, shortest=2, least_click=0)

In [ ]:
stats_info(tr, 'train (item_idx -1 필터링 및 재클렌징 후)')
stats_info(val, 'valid (item_idx -1 필터링 및 재클렌징 후)')
stats_info(test, 'test (item_idx -1 필터링 및 재클렌징 후)')

* train (item_idx -1 필터링 및 재클렌징 후) Set Stats Info
	 Events: 989344
	 Sessions: 6039
	 Items: 3416
	 First Time : 2000-04-25 23:05:32
	 Last Time : 2002-08-12 13:53:29

* valid (item_idx -1 필터링 및 재클렌징 후) Set Stats Info
	 Events: 5150
	 Sessions: 195
	 Items: 1929
	 First Time : 2002-08-13 00:40:06
	 Last Time : 2002-11-20 16:38:40

* test (item_idx -1 필터링 및 재클렌징 후) Set Stats Info
	 Events: 5032
	 Sessions: 219
	 Items: 1880
	 First Time : 2002-11-21 01:02:06
	 Last Time : 2003-02-28 17:49:50



In [ ]:
tr

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,2000-12-31 22:00:19,0
22,1,1270,5,2000-12-31 22:00:55,1
27,1,1721,4,2000-12-31 22:00:55,2
37,1,1022,5,2000-12-31 22:00:55,3
24,1,2340,3,2000-12-31 22:01:43,4
...,...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29,1248
999988,6040,1921,4,2001-08-10 14:41:04,370
1000172,6040,1784,3,2001-08-10 14:41:04,89
1000167,6040,161,3,2001-08-10 14:41:26,464


In [ ]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # UserId를 기준으로 인덱싱

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

In [ ]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,2000-12-31 22:00:19,0
22,1,1270,5,2000-12-31 22:00:55,1
27,1,1721,4,2000-12-31 22:00:55,2
37,1,1022,5,2000-12-31 22:00:55,3
24,1,2340,3,2000-12-31 22:01:43,4
36,1,1836,5,2000-12-31 22:02:52,5
3,1,3408,4,2000-12-31 22:04:35,6
7,1,2804,5,2000-12-31 22:11:59,7
47,1,1207,4,2000-12-31 22:11:59,8
0,1,1193,5,2000-12-31 22:12:40,9


In [ ]:
tr_dataset.click_offsets

array([     0,     53,    182, ..., 988880, 989003, 989344])

In [ ]:
tr_dataset.session_idx

array([   0,    1,    2, ..., 6036, 6037, 6038])

In [ ]:
class FullSequenceDataLoader:
    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size
    
    
    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            target_rating (B,): Ratings of the target items for weighting.
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                target_rating = self.dataset.df['Rating'].values[start + i + 1] # Rating은 다음 아이템에 대한 평점입니다.
                yield inp, target, target_rating, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.

        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더함
        mask = np.arange(self.batch_size)[(end - start) == 1]
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것
            new_session = last_session + i
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록
        return start, end, mask, last_session, finished

In [ ]:
tr_data_loader = FullSequenceDataLoader(tr_dataset, batch_size= 190)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,2000-12-31 22:00:19,0
22,1,1270,5,2000-12-31 22:00:55,1
27,1,1721,4,2000-12-31 22:00:55,2
37,1,1022,5,2000-12-31 22:00:55,3
24,1,2340,3,2000-12-31 22:01:43,4
36,1,1836,5,2000-12-31 22:02:52,5
3,1,3408,4,2000-12-31 22:04:35,6
7,1,2804,5,2000-12-31 22:11:59,7
47,1,1207,4,2000-12-31 22:11:59,8
0,1,1193,5,2000-12-31 22:12:40,9


In [ ]:
iter_ex = iter(tr_data_loader)

In [ ]:
inputs, labels, target_ratings, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [   0   53   65   54   56  385    1   54  514  429   53  829   37  860
  870  927  945   93  985   16  944   78  218   89 1046   65  552    9
 1468  805 1491   89  799 1592  117 1205 1083  599    2   81   32    9
  125 1600 1096 1735    9   37  163 1115  267   65   93    9  453   93
 2003 1914   11  593 1180  267  325   53  597   16  599 2137  183  237
 2060   54   78    2 2169 1787 2202  235  661  134 2161  385   78  125
 1159  408 2234   91  502  126 1400 1105  163 1068   81    9  386 1499
  507  668  491  739  554  552  134   91 1022    1    1  546 1090 2329
  491 1774  343 1671  186 1234  664   11   53   16   28   93 1633   93
    1 1046 1910  267   54 2323   62  901 2260    9  165  501    9   15
 1258  155   37  127 1391   78   49  109  326  316  517 1054 1340   78
  874  799  552  386  114   24   81    2 1173   24   53   91  483  685
  611  267   82 1505  154  688  134  258   11    9   78 1468  405  514
   25   78  163 2018   65  385 1440    2]
Label It

In [ ]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU # type: ignore
from tensorflow.keras.losses import categorical_crossentropy # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore
from tqdm import tqdm 

In [ ]:
def create_model(args):
    # input: (batch, sequence_length, num_items)
    # inputs = Input(shape=(None, args.num_items))  # 시퀀스 길이는 가변, 배치 크기도 가변
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k
        self.max_rating = data['Rating'].max() # Rating의 최대값 저장 

args = Args(tr, val, test, batch_size=190, hsz=100, drop_rate=0.1, lr=0.001, epochs=5, k=20)

In [ ]:
model = create_model(args)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (190, 1, 3416)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU (GRU)                       │ [(190, 100), (190,     │     1,055,400 │
│                                 │ 100)]                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (190, 100)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (190, 3416)            │       345,016 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,400,416 (5.34 MB)

 Trainable params: 1,400,416 (5.34 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = FullSequenceDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['UserId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)

    
        for feat, target, target_rating, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)
            # target_rating은 평점이므로 0~1 사이로 변환
            # sample_weights는 평점에 따라 가중치를 부여합니다. (평점이 높을수록 중요하다고 가정)
            sample_weights = target_rating / args.max_rating

            result = model.train_on_batch(input_ohe, target_ohe, sample_weight=sample_weights)
            tr_loader.set_postfix(train_loss=result[0], accuracy=result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k, data_name='valid')

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:.4f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:.4f}")

def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옴
    current_hidden_state_tensor = gru_layer.states[0] # 현재 hidden state tensor를 가져옴
    # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
    hidden_states_np = current_hidden_state_tensor.numpy()
    # mask된 인덱스에 해당하는 행을 0으로 초기화
    for elt in mask:
        hidden_states_np[elt, :] = 0
    # 초기화된 numpy array를 다시 tensor로 변환
    # gru_layer.reset_states(states=hidden_states_np)과 동일
    current_hidden_state_tensor.assign(hidden_states_np)  

# 평가 메트릭스를 계산하는 함수
def get_metrics(data, model, args, k: int, data_name: str = "Unknown"):
    recall_list, mrr_list = [], []
    
    # 평가를 시작하기 전에 모델의 GRU 레이어 상태를 초기화
    gru_layer = model.get_layer(name='GRU')
    gru_layer.reset_states() # 평가 시작 시 GRU 레이어의 모든 상태를 초기화합니다.

    print(f"\n--- {data_name} 평가 시작 ---")
    print(f"평가 데이터셋 크기: {len(data)}")
    print(f"평가 세션 수: {data['UserId'].nunique()}")
    
    dataset = SessionDataset(data)
    
    if dataset.session_idx.size == 0:
        print(f"평가 데이터({data_name})에 유효한 세션이 없습니다.")
        return 0.0, 0.0

    processed_sessions_count = 0
    total_predictions_made = 0

    for i in tqdm(range(dataset.session_idx.size), desc=f'평가 중 ({data_name})', mininterval=1):
        session_start_idx = dataset.click_offsets[i]
        session_end_idx = dataset.click_offsets[i + 1]
        
        session_events = dataset.df['item_idx'].values[session_start_idx:session_end_idx]

        if len(session_events) < 2:
            continue

        processed_sessions_count += 1

        for j in range(len(session_events) - 1):
            input_item = session_events[j]
            target_item = session_events[j+1]
            
            input_ohe = to_categorical([input_item], num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)

            pred = model.predict(input_ohe, batch_size=1, verbose=0)
            
            pred_arg = tf.argsort(pred[0], direction='DESCENDING').numpy()
            
            recall_list.append(recall_k(pred_arg, target_item, k))
            mrr_list.append(mrr_k(pred_arg, target_item, k))
            total_predictions_made += 1
            
        # 한 세션의 모든 단계를 처리한 후, 다음 세션을 위해 GRU 레이어의 hidden state를 초기화
        # model.predict(..., batch_size=1)로 호출했으므로, GRU 레이어의 상태는 (1, hsz) 형태를 가집니다.
        # 따라서 해당 상태 텐서의 모든 값을 0으로 설정하여 초기화
        current_hidden_state_tensor = gru_layer.states[0] # GRU 레이어의 현재 hidden state 텐서
        # 해당 텐서를 0으로 채워진 동일한 형태의 넘파이 배열로 변환 후, 다시 텐서에 할당
        current_hidden_state_tensor.assign(np.zeros_like(current_hidden_state_tensor.numpy()))

    print(f"평가를 위해 처리된 세션 수: {processed_sessions_count}")
    print(f"평가를 위해 생성된 총 예측 수: {total_predictions_made}")
    
    if not recall_list: 
        print("Recall 리스트가 비어있습니다. 유효한 예측이 없거나 평가되지 않았습니다.") 
        return 0.0, 0.0
    
    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    print(f"--- {data_name} 평가 완료 ---")
    return recall, mrr

In [ ]:
train_model(model, args)

Train:  97%|█████████▋| 5001/5175 [02:09<00:04, 38.69it/s, accuracy=0.036337294, train_loss=4.296075] 



--- valid 평가 시작 ---
평가 데이터셋 크기: 5150
평가 세션 수: 195


평가 중 (valid): 100%|██████████| 195/195 [04:45<00:00,  1.46s/it]


평가를 위해 처리된 세션 수: 195
평가를 위해 생성된 총 예측 수: 4955
--- valid 평가 완료 ---
	 - Recall@20 epoch 1: 0.1320
	 - MRR@20    epoch 1: 0.0358


Train:  97%|█████████▋| 5001/5175 [02:17<00:04, 36.42it/s, accuracy=0.04154248, train_loss=4.20369]   


...

평가를 위해 처리된 세션 수: 195
평가를 위해 생성된 총 예측 수: 4955
--- valid 평가 완료 ---
	 - Recall@20 epoch 4: 0.1332
	 - MRR@20    epoch 4: 0.0376


Train:  97%|█████████▋| 5001/5175 [02:08<00:04, 38.93it/s, accuracy=0.051152777, train_loss=4.04966]  



--- valid 평가 시작 ---
평가 데이터셋 크기: 5150
평가 세션 수: 195


평가 중 (valid): 100%|██████████| 195/195 [04:45<00:00,  1.46s/it]

평가를 위해 처리된 세션 수: 195
평가를 위해 생성된 총 예측 수: 4955
--- valid 평가 완료 ---
	 - Recall@20 epoch 5: 0.1334
	 - MRR@20    epoch 5: 0.0378


In [ ]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20, data_name='test')
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)


--- test 평가 시작 ---
평가 데이터셋 크기: 5032
평가 세션 수: 219


평가 중 (test): 100%|██████████| 219/219 [04:35<00:00,  1.26s/it]

평가를 위해 처리된 세션 수: 219
평가를 위해 생성된 총 예측 수: 4813
--- test 평가 완료 ---
	 - Recall@20: 0.177021
	 - MRR@20: 0.054943

